In [1]:
import pandas as pd
import numpy as np
import random

# implementing random seed for control
np.random.seed(42)

# school names for consistency 
schools = ['Oakland Middle School',
    'Siegel Middle School',
    'Whitworth-Buchanan Middle School',
    'Christiana Middle School',
    'Smyrna Middle School',
    'Stewarts Creek Middle School',
    'Rockvale Middle School',
    'Rocky Fork Middle School',
    'Blackman Middle School',
    'Thurman Francis Arts Academy',
    'Rock Springs Middle School',
    'LaVergne Middle School'
]

# Decisions

### New Capacity_Report.csv

Updated Features (5)
<ul>
    <li><i>"School"</i>, unchanged.</li></br>
    <li><strong><i>"Status"</i></strong>, updated with now 3 options that describe ONLY volume of 8th graders, NOT whether Pathways demand is accomodated. 
    <ol>
        <li>'Sufficient' indicates that the volume of students (VOS) can be housed with the alotted rooms.</li>
        <li>'Suff. w/ All' indicates that the VOS can be accomodated only by including the rooms labeled (if needed) on the county provided planning document.</li>
        <li>'Insufficient' indicates not enough classroom space has been provided by the school.</li>
    </ol>
    </li></br>
    <li><strong><i>"Assigned Default Capacity"</i></strong>, formerly "Assigned Capacity", represents total volume of rooms excluding any labeled (if needed).</li></br>
    <li><strong><i>"Assigned Max Capacity"</i></strong>, added, represents the total classroom volume provided by school.</li></br>
    <li><i>"8th Graders"</i>, unchanged, volume according to school's skyward roster.</li></br>
    <li><s><i>"Number of Large Rooms"</i></s>, removed, in the updated files from the county, each school provides a near-unique capacity vector with unspecified number of sizes instead of the previous binary split between 50/35 volume for rooms.</li></br>
    <li><s><i>"Number of Small Rooms"</i></s>, removed, see preceding bullet.</li></br>
</ul>

### Creating a df of capacity vectors per school
<strong>Issue:</strong> the current planning document from the county does not have an updated list of Pathways of Study per shcool. It might be different that previously, but trying to solve the last iterations problem might not be worth it if the new iteration will be addressing a different situation. </br>

<u>Current proposed export</u>: a .csv document with two features: school, and a dict-like object that represents the schools capacity vector along with room assignment. </br>

Motivation: could assign correct rooms on the forward part of the algorithm without an extra lift, and eliminate the need to update the planning document after-the-fact with new assignments.  

# Next Steps

Need to implement metrics for measuring effectiveness of meeting POS demand. As of now, all schools have provided enough space without their (if needed) rooms. In several cases, those rooms are very large (50+ occupancy). Yes, they can house the students without them, but are they serving the students effectively? 

In [ ]:
""" 
Objectives:
(1) Implement sufficient cpacity clause into new capacity_report.py
(2) (A)Create and (B)store capcity vector per school
(3) Brainstorm handling for TF/SW combo...

Citation: 
- https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value, 
    provided insight into efficiently utilizing lambda syntax to sort vector_dict objects

"""

In [2]:
# capacity report features
capacity_report = {
    #(A-E)
    'School':[],
    'Status':[],
    'Assigned Default Capacity':[],
    'Assigned Max Capacity':[],
    '8th Graders':[],
    # 'Number of Large Rooms':[],
    # 'Number of Small Rooms':[],
}

capacity_vector_report = {
    #(F-G)
    'School':[],
    'Capacity_Vector':[],
}

In [3]:
# building capacity report AND building capacity vector
for school in schools:
    # read in planning document
    # NOTE: currently reading in new doc from desktop until county finalizes docs
    p = '../../../../Desktop/From RuCo/planning.xlsx'
    try:
        if school == 'Whitworth-Buchanan Middle School':
            planning_df = pd.read_excel(p,sheet_name='Whitworth-Buchanan Middle Schoo')
        else:
            planning_df = pd.read_excel(p,sheet_name=school)
    except:
        continue
    #(A,F)
    capacity_report['School'].append(school)
    capacity_vector_report['School'].append(school)

    #(D)
    max_cap = sum(list(planning_df['Max Room Capacity']))
    capacity_report['Assigned Max Capacity'].append(max_cap)

    #(E)
    # read in skyward roster for length, NOTE: currently reading from desktop folder
    sky_p = f'../../../../Desktop/From RuCo/Skyward/Skyward_{school}.xlsx'
    sky_df = pd.read_excel(sky_p)
    volume = len(sky_df.drop_duplicates("Student's School Email")) # necessary! 
    capacity_report['8th Graders'].append(volume)
    
    #(C), (G), pt. 1 set for default rooms
    capacity_vector = {}
    default_rooms, if_needed_rooms = [], [] # need this for building capacity vector if not using all rooms
    default_capacity = 0
    for room in list(planning_df['MS Room Number']):
        cap = planning_df.loc[planning_df['MS Room Number'] == room, 'Max Room Capacity'].values[0]
        if "(if needed)" not in str(room):
            default_rooms.append(room)
            default_capacity += cap
            capacity_vector[room] = cap
        else:
            if_needed_rooms.append(room)
    capacity_report['Assigned Default Capacity'].append(default_capacity)
    
    #(B) - determining status, NOTE: this does not reflect capcity to meet demand for pathways of study
    diff = default_capacity - volume
    abs_diff = max_cap - volume
    if diff > 0:
        status = ['Sufficient',0]
    elif diff < 0 and abs_diff < 0:
        status = ['Insufficient',1]
    else:
        status = ['Suff. w/ All',2]
    capacity_report['Status'].append(status[0])

    #(G), pt. 2 - checking if we need to add conditionally assigned rooms
    if status[1] != 0: # NOT use only default
        for room in if_needed_rooms:
            cap = planning_df.loc[planning_df['MS Room Number'] == room, 'Max Room Capacity'].values[0]
            capacity_vector[room] = cap

    #(G), pt. 3 sorting capacity vector in descending order, gets sorted on unpacking
    # capacity_vector = {k: v for k, v in sorted(capacity_vector.items(), key=lambda item: item[1])}
    capacity_vector_report['Capacity_Vector'].append(capacity_vector)
    

In [4]:
pd.DataFrame(capacity_report, columns=list(capacity_report.keys()))

,School,Status,Assigned Default Capacity,Assigned Max Capacity,8th Graders
0,Oakland Middle School,Sufficient,475,475,405
1,Siegel Middle School,Sufficient,655,655,402
2,Whitworth-Buchanan Middle School,Sufficient,440,440,270
3,Christiana Middle School,Insufficient,0,0,331
4,Smyrna Middle School,Sufficient,430,480,303
5,Stewarts Creek Middle School,Sufficient,390,390,343
6,Rockvale Middle School,Sufficient,807,897,502
7,Rocky Fork Middle School,Sufficient,340,430,238
8,Blackman Middle School,Sufficient,835,835,561
9,Rock Springs Middle School,Sufficient,450,450,401


In [6]:
pd.DataFrame(capacity_vector_report, columns=list(capacity_vector_report.keys())).to_csv('../YouScienceData/Reports/capacity_vectors.csv')

In [8]:
def extract_capacity_vector(school):
    # read in capacity vector object 
    df = pd.read_csv('../YouScienceData/Reports/capacity_vectors.csv')
    df.drop("Unnamed: 0", axis=1, inplace=True)

    # inital object is long str
    vector_str = df.loc[df.School == school, "Capacity_Vector"].values[0]
    if vector_str == '{}':
        print(f'{school} missing data.')
        return 0
    # transforms it into list of room: cap strings
    vector_list = vector_str.split(', ')

    # transforming into actionable dictionary vector
    vector_dict = {}
    for item in vector_list:
        k, v = item.split(': ')
        # checking for first item to have '{'
        if '{' in k:
            k = k[1:]
        # checking for last item to have '}' 
        if '}' in v:
            v = v[:-1]
        try:
            vector_dict[int(k)] = int(v)
        except ValueError:
            # shave off quotations 
            vector_dict[k[1:-1]] = int(v)
    
    return {k: v for k, v in sorted(vector_dict.items(), key=lambda item: item[1], reverse=True)}


In [12]:
# trying over all schools
all_cap_vectors = {}
for school in schools:
    if school == 'Thurman Francis Arts Academy':
        continue
    
    all_cap_vectors[school] = extract_capacity_vector(school=school)

Christiana Middle School missing data.


In [13]:
ran = list(all_cap_vectors.keys())
for school in schools:
    if school not in ran:
        print(school)

Thurman Francis Arts Academy


In [14]:
schools

['Oakland Middle School',
 'Siegel Middle School',
 'Whitworth-Buchanan Middle School',
 'Christiana Middle School',
 'Smyrna Middle School',
 'Stewarts Creek Middle School',
 'Rockvale Middle School',
 'Rocky Fork Middle School',
 'Blackman Middle School',
 'Thurman Francis Arts Academy',
 'Rock Springs Middle School',
 'LaVergne Middle School']

In [21]:
all_cap_vectors[schools[6]]

{'Library': 60,
 807: 55,
 805: 52,
 800: 40,
 801: 40,
 802: 40,
 803: 40,
 804: 40,
 806: 40,
 809: 40,
 810: 40,
 811: 40,
 812: 40,
 813: 40,
 814: 40,
 815: 40,
 816: 40,
 819: 40,
 820: 40}

In [ ]:
test_dict = {
    'str1':25,
    'str2':15,
    'str3':103,
    'str4':11,
    'str5':1000,
}

test_dict = {k: v for k, v in sorted(test_dict.items(), key=lambda item: item[1], reverse=True)}
test_dict

# Checking Skyward Docs. 

Checking format. Found exception with Oakland Middle School


In [ ]:
for school in schools:
    sky_p = f'../../../../Desktop/From RuCo/Skyward/Skyward_{school}.xlsx'
    sky_df = pd.read_excel(sky_p)
    print(school, len(sky_df), len(sky_df.drop_duplicates("Student's School Email")))